# Capstone Project
This notebook and repo will be used for IBM's data science professional capstone course on coursera. 

In [2]:
# import dependencies
import pandas as pd 
import numpy as np

## Table of contents
* [Introduction: Business Problem](#introduction)
* [Data](#data)
* [Methodology](#methodology)
* [Analysis](#analysis)
* [Results and Discussion](#results)
* [Conclusion](#conclusion)

## Introduction: Business Problem <a name="introduction"></a>

Car accidents happen every day all over the world. Due to the energy involved in these collisions, there is a high chance that someone is seriously injured in one of these accidents. To properly treat these victims, medical personnel need to be ready to administer aid immediately. Having an estimation of the severity of the collision can help first responders prepare treatment and potentially save lives. This project is meant to help serve first responders so that they can have an estimation of the severity of the accident before arriving on the scene to know what type of aid is needed. 

## Data <a name="data"></a>